# 02 - Les Acteurs

In [23]:
import logging
import random
import time
from random import randint
from typing import Dict, List

import numpy as np
import ray
from ray import ObjectRef

In [ ]:
ray.init(
    ignore_reinit_error=True,
    logging_level=logging.ERROR,
)

Dans ce notebook, on va essayer d'explorer quelques patterns que l'on peut utiliser avec les **Actors**.

## La Remote Class

Pour définir notre premier **Actor**, nous allons définir une classe avec le décorateur `@ray.remote`.

Cette classe sera un `ParameterServer`, un exemple courant en machine learning où une instance maitresse (**La Remote Class**) peut mettre à jour les gradients provenant d'autres processus, par exemple des workers qui calculent les gradients individuels sur des batches.

<img src="../images/parameter_server.png" width="40%" height="20%">

In [3]:
@ray.remote
class ParameterSever:
    def __init__(self):
        self.params = np.zeros(10)

    def get_params(self):
        return self.params

    def update_params(self, gradients):
        self.params -= gradients

On va ensuite définir worker, une task qui va modéliser un calcul des gradients typique du machine learning, et qui va les envoyer au `ParameterServer`.

In [4]:
@ray.remote
def worker(parameter_server: ParameterSever):
    # Modélise les epoch
    for i in range(100):
        # Modélise le calcul des gradients
        time.sleep(1.5)
        grad = np.ones(10)
        parameter_server.update_params.remote(grad)

On démarre notre actor `ParameterServer`. Il va être schedulé comme un process sur un worker Ray.

In [ ]:
parameter_server = ParameterSever.remote()
parameter_server

Ça tourne ! Vérifions les poids initiaux stockés sur notre actor :

In [ ]:
print(f"Initial params: {ray.get(parameter_server.get_params.remote())}")

Maintenant, on va créer trois workers distincts représentant nos tâches de Machine Learning qui calculent des gradients.

In [ ]:
[worker.remote(parameter_server) for _ in range(3)]

Maintenant, itérons dans une boucle et regardons l'évolution de nos poids dans le `ParameterServer` pendant que les workers s'exécutent indépendamment et mettent à jour les gradients.

In [ ]:
for _i in range(20):
    print(f"Updated params: {ray.get(parameter_server.get_params.remote())}")
    time.sleep(1)

## L'arbre d'Actors

L'`Arbre des Actors` ou le `Tree of Actors` est un pattern courant utilisé dans les bibliothèques Ray comme `Ray Tune`, `Ray Train` et `RLlib` pour entraîner des modèles en parallèle ou effectuer une [optimisation des hyperparamétres (HPO)](https://en.wikipedia.org/wiki/Hyperparameter_optimization) distribuée.

Dans ce modèle, un arbre d'acteurs, les feuilles correspondent à des actors. Les collections d'actors sont gérées des `supervisor`. Ce pattern peut être utilisé lorsqu'on souhaite entraîner plusieurs modèles simultanément tout en étant capable de sauvegarder des checkpoints et d'inspecter les états.

```
                                                                          
                            +---------+                                   
                            |  Driver |                                   
                            +---------+                                   
                                 |                                        
                      +----------------------+                            
                      |                      |                            
             +----------------+     +----------------+                    
             | Supervisor     |     | Supervisor     |                    
             | Actor          |     | Actor          |                    
             +----------------+     +----------------+                    
                      |                      |                            
               +-----------+          +------------+                      
               |           |          |            |                      
           +-------+   +-------+   +-------+   +-------+                  
           | Worker|   | Worker|   | Worker|   | Worker|                  
           | Actor |   | Actor |   | Actor |   | Actor |                  
           +-------+   +-------+   +-------+   +-------+                  
```


Dans un premier temps, on peut creer une classe qui représente un `Model`, il ne s'agit pas d'un actor car ici on ajoute juste les fonctionnalités spécifiques au modèle, comme par exemple sa manière de s'entrainer.

In [9]:
class Model:
    def __init__(self, model: str):
        self._model = model

    def train(self):
        time.sleep(1)  # Modélise les calculs


def model_factory(model: str):
    return Model(model)

Ensuite, on va créer un actor, qui va représenter notre worker, avec ses différents états, et ses différentes fonctionnalités.

In [10]:
STATES = ["RUNNING", "DONE"]


@ray.remote
class Worker(object):
    def __init__(self, model: str):
        self._model = model

    def state(self) -> str:
        return random.choice(STATES)

    def work(self) -> None:
        model_factory(self._model).train()

Ces Workers vont être déployés et controlés par les Supervisors.

In [11]:
@ray.remote
class Supervisor:
    def __init__(self) -> None:
        self.workers = [
            Worker.remote(name)
            for name in ["Logical Regression", "Classification", "Neural Metworks"]
        ]

    def work(self) -> None:
        [w.work.remote() for w in self.workers]

    def terminate(self) -> None:
        [ray.kill(w) for w in self.workers]

    def state(self) -> List[str]:
        return ray.get([w.state.remote() for w in self.workers])

On peut ainsi lancer nos entrainements de la manière suivante :

In [ ]:
supervisor = Supervisor.remote()

supervisor.work.remote()

In [ ]:
while True:
    states = ray.get(supervisor.state.remote())
    print(states)
    result = all("DONE" == e for e in states)
    if result:
        supervisor.terminate.remote()
        ray.kill(supervisor)
        break

## Les Messages

Maintenant, nous allons voir un nouveau pattern pour les systèmes de message. Il s'agit du `MessageActor`.

In [14]:
@ray.remote
class MessageActor(object):
    def __init__(self):
        self.messages = []

    def add_message(self, message):
        self.messages.append(message)

    def get_and_clear_messages(self):
        messages = self.messages
        self.messages = []
        return messages

In [15]:
message_actor = MessageActor.remote()

Définissons une task qui boucle et envoie des messages à `MessageActor`. Elle va lui envoyer un identifiant avec un message associé.

In [16]:
@ray.remote
def worker(message_actor, j):
    for i in range(10):
        time.sleep(1)
        message_actor.add_message.remote(f"Message {i} from worker {j}.")

On va démarrer en arrière plan trois tâches qui doivent communiquer avec le `MessageActor`.

In [ ]:
[worker.remote(message_actor, j) for j in range(3)]

Pour les récupérer, on va executer une boucle qui périodiquement va appeler la fonction `get_and_clear_messages`.

In [ ]:
for _ in range(10):
    new_messages = ray.get(message_actor.get_and_clear_messages.remote())
    print("New messages\n:", new_messages)
    time.sleep(1)

----

### Excercice : Gestion d'une flotte de taxis

À vous de jouer !

Vous êtes chargé de modéliser un service de taxi utilisant Ray pour gérer une flotte.

Vous devez simuler une flotte de taxis où chaque taxi gagne de l'argent en fonction des courses qu'il effectue. Son tarif est de 1.25 dollar par seconde, mais il doit reverser une commission de 25 cents par seconde au dispatcher. Le dispatcher doit tenir un compte de l'argent total collecté, et doit distribuer les missions aux taxis.

Quelques notes techniques avant de démarrer.

- Comment passer un actor, à un autre actor ?
   - Attention, `self` est un mot clef python, et il n'est pas vraiment compatible avec `Ray`... Si vous devez passer la référence de l'actor courant vers un autre actor il faudra utiliser :
   - `ray.get_runtime_context().current_actor`

- Si j'appelle deux fonctions d'un actor... Est-ce que les appels sont parallélisés ?
   - Non, pourquoi le serait-ils ?

**👇 À COMPLETER**


Vous allez implémenter une classe Taxi en tant qu'acteur Ray. Cette classe représente un taxi individuel dans un système de gestion de flotte. Elle doit permettre d'attribuer des courses et de suivre les gains du chauffeur.


<br/>

**Attributs de la classe**

Votre classe Taxi doit inclure les attributs suivants :

 - `message_actor` : Une référence à l'acteur MessageActor, utilisée pour enregistrer des messages sur les actions du taxi.
 - `dispatcher` : Une référence à l'acteur Dispatcher, permettant au taxi de notifier la fin d'une course.
 - `name` : Une chaîne de caractères représentant le nom du chauffeur.
 - `cash` : Un float représentant les gains nets du taxi.
 - `state` : Une chaîne de caractères indiquant l'état actuel du taxi (`DISPONIBLE` ou `OCCUPE`).

<br/>


**Méthodes à implémenter**

`__init__(name, message_actor, dispatcher)`

- Initialisez les attributs message_actor, dispatcher, name, cash, et state.
- Le taxi commence avec un cash initial de 0.0 et dans l'état DISPONIBLE.


`take_ride(duration)`

- Changez l'état du taxi à OCCUPE.
- Simulez la durée de la course en utilisant time.sleep(duration).
- Calculez les gains nets pour la course.
- Appelez la méthode ride_complete.remote() du dispatcher pour notifier la fin de la course et envoyer la commission correspondante.
- Revenez à l'état DISPONIBLE une fois la course terminée.


<br/>

**Contraintes**

- Les appels au dispatcher, tels que ride_complete, doivent être effectués avec .remote() pour garantir qu'ils sont asynchrones et ne bloquent pas le taxi.
- La méthode `take_ride` doit s'exécuter sans interaction directe avec le système principal (pas de retour ou ray.get).


In [ ]:
DISPONIBLE, OCCUPE = "disponible", "occupe"
PRICE_PER_SECOND = 1.25
COMMISSION_PER_SECOND = 0.25


@ray.remote
class Taxi:
    message_actor: MessageActor
    dispatcher: "Dispatcher"
    name: str
    cash: float
    state: str

    def __init__(
        self, name, message_actor: MessageActor, dispatcher: "Dispatcher"
    ) -> None:
        # SOLUTION
        self.message_actor = message_actor
        self.dispatcher = dispatcher
        self.name = name
        self.cash = 0.0
        self.state = DISPONIBLE

    def take_ride(self, duration: int) -> None:
        # SOLUTION
        self.state = OCCUPE
        time.sleep(duration)
        self.cash += duration * (PRICE_PER_SECOND - COMMISSION_PER_SECOND)
        self.dispatcher.ride_complete.remote(
            self.name, duration * COMMISSION_PER_SECOND
        )
        self.state = DISPONIBLE

**👇 À COMPLETER**

Vous allez implémenter une classe Dispatcher en tant qu'acteur Ray. Cette classe aura pour rôle de gérer une flotte de taxis et leur état.

<br/>

**Attributs de la classe**

Votre classe devra inclure les éléments suivants :

 - Un acteur pour la gestion des messages (message_actor).
 - Une variable représentant l'argent total collecté via les commissions.
 - Une structure pour suivre l'état des chauffeurs (noms et états).
 - Une liste contenant les taxis gérés par le dispatcher.

La liste des employés est :
 - Daniel
 - Petra
 - Gilbert
 - Emilien


<br/>


**Méthodes à implémenter**:


`__init__`
- Initialisez les attributs nécessaires, créez la flotte de taxis en passant les informations nécessaires (nom du chauffeur, message_actor, etc.), et configurez le système de gestion des états.

`assign_ride(duration)`
- Attribuez une course à un chauffeur disponible. Signaler cette attribution au `MessageActor`.
- Si aucun taxi n'est disponible, signalez le également.

`ride_complete(driver, commission)`
- Lorsque la course est terminée, mettez à jour les états des chauffeurs et ajoutez la commission collectée au total. Signaler cette fin de course au `MessageActor`.

`get_cash()`
- Retournez l'argent total collecté par le dispatcher.


<br/>


**Contraintes et Système d'États**

- La gestion des états des taxis doit être basée uniquement sur une structure interne (pas de ray.get pour vérifier l'état d'un taxi en cours de course, car s'il est en train de faire une course, il faudrait qu'il la finisse pour répondre).
- Chaque méthode doit respecter l'asynchronisme de Ray, avec l'utilisation appropriée de .remote().

In [ ]:
@ray.remote
class Dispatcher:
    message_actor: MessageActor
    cash: float
    staff: Dict[str:str]  # {employee: status}
    taxis: List[ObjectRef]

    def __init__(self, message_actor) -> None:
        # SOLUTION
        self.message_actor = message_actor
        self.cash = 0.0
        self.staff = {
            name: DISPONIBLE for name in ["Daniel", "Petra", "Gilbert", "Emilien"]
        }
        self.taxis = [
            Taxi.remote(name, message_actor, ray.get_runtime_context().current_actor)
            for name in self.staff.keys()
        ]

    def assign_ride(self, duration) -> None:
        # SOLUTION
        for (driver, status), taxi in zip(self.staff.items(), self.taxis):
            if status == DISPONIBLE:
                self.message_actor.add_message.remote(
                    f"Course de {duration} secondes envoyée à {driver}"
                )
                self.staff[driver] = OCCUPE
                taxi.take_ride.remote(duration)
                return
        self.message_actor.add_message.remote(f"Tous les taxis sont occupés")

    def ride_complete(self, driver, commission: float) -> None:
        # SOLUTION
        self.message_actor.add_message.remote(
            f"Commission de {commission} reçu de la part de {driver}"
        )
        self.cash += commission
        self.staff[driver] = DISPONIBLE

    def get_cash(self):
        # SOLUTION
        return self.cash

**👇 À COMPLETER**

Vous allez implémenter une classe Simulation en tant qu'acteur Ray. Cette classe sera responsable de simuler une journée de travail pour un dispatcher et sa flotte de taxis.

<br/>

**Attributs de la classe**

Votre classe Simulation doit contenir :
- Une référence à un acteur Dispatcher qui sera responsable de gérer les taxis et leurs états.

<br/>

**Méthodes à implémenter**

`__init__(dispatcher)`
- Recevez une référence à l'acteur Dispatcher.
- Stockez cette référence dans un attribut pour pouvoir interagir avec le dispatcher au cours de la simulation.

`run()`
- Simulez une série d'événements sur une durée définie (par exemple, une journée de travail fictive).
- À chaque itération :
    - Générez une durée aléatoire pour une course.
    - Appelez la méthode assign_ride de l'acteur Dispatcher avec cette durée.
    - Attendez un temps fixe avant de passer à la prochaine itération pour simuler l'écoulement du temps.

`get_dispatcher()`
- Retournez la référence au dispatcher, permettant d'inspecter son état ou d'interagir avec lui en dehors de la simulation.

<br/>

**Contraintes**
- La simulation doit être non bloquante pour d'autres processus Ray. Cela signifie que toutes les interactions avec le Dispatcher doivent se faire via .remote().
- Le déroulement de la simulation doit imiter une journée réaliste avec des pauses régulières entre les courses.

In [28]:
@ray.remote
class Simulation:
    def __init__(self, dispatcher):
        # SOLUTION
        self.dispatcher = dispatcher

    def run(self):
        # SOLUTION
        for _ in range(10):
            self.dispatcher.assign_ride.remote(random.randint(1, 5))
            time.sleep(1)

    def get_dispatcher(self):
        # sOLUTION
        return self.dispatcher

In [ ]:
message_actor = MessageActor.remote()
dispatcher = Dispatcher.remote(message_actor)
simulation = Simulation.remote(dispatcher)
simulation.run.remote()

for _ in range(15):
    messages = ray.get(message_actor.get_and_clear_messages.remote())
    print(messages)
    time.sleep(1)

total_commission = ray.get(dispatcher.get_cash.remote())
print(f"Commission totale collectée {total_commission}")

In [20]:
# ray.shutdown()